## Setup

Add any imports needed here

Mount Google Drive

In [14]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install datasets
!pip -q install transformers
!pip -q install -U peft
!pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
!pip -q install rouge

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui


In [2]:
import numpy as np
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
from bs4 import BeautifulSoup

drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS6220 Folder #add direct access to the folder if you get an error in this cell
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

import helpers
import importlib

Mounted at /content/drive


## **IBM Python Dataset**

## Dataset

We use a different code dataset which we then partition into artificial "repositories" using PyTorch dataloader

In [3]:
## TODO: Diana
## Download Dataset
%cd /content/drive/.shortcut-targets-by-id/1ZKIbzAJY4RXvlu64yVQt4OckCxyNqRJ6/CS6220 Folder

/content


The next approach uses the Scripts_Project_CodeNet_Python800 folder. It contains .py scripts. Each script has multiple chunks of code (combined from the previous structure, where in each folder we had multiple scripts). The delimiter between chunk and chunk is a string of hashtags. (#####################################################)

In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import random


class CustomDataset(Dataset):

    def __init__(self, folder_path, partition, delimiter="#", chunk_size=100): #initializes the dataset with the folder path, delimiter, and chunk size
        self.folder_path = folder_path
        self.delimiter = delimiter
        self.chunk_size = chunk_size

        self.script_files = [f for f in os.listdir(folder_path) if f.endswith(".py")] # Get a list of all Python scripts in the specified folder with a ".py" extension
        if partition == 1:
          self.script_files = self.script_files[:10]
        else:
          self.script_files = self.script_files[10:20]

        self.data = []

        for idx in range(len(self.script_files)):
          script_file_path = os.path.join(self.folder_path, self.script_files[idx])
          with open(script_file_path, 'r', encoding='utf-8') as file:
              code_samples = file.read().split(self.delimiter)
              code_samples = [i for i in code_samples if len(i) > 5]
              self.data.extend(code_samples)
        print(f'Code samples in partition {partition}: {len(self.data)}')

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx): # method to get a specific item from the dataset (returns a chunk of code)
        return self.data[idx]


In [5]:
folder_path = "/content/drive/.shortcut-targets-by-id/1ZKIbzAJY4RXvlu64yVQt4OckCxyNqRJ6/CS6220 Folder/Scripts_Project_CodeNet_Python800"
delimiter = "#####################################################"
chunk_size = 1
batch_size = 16

# Create instances of the dataset with specified file indices
dataset_1 = CustomDataset(folder_path, 1, delimiter, chunk_size)
dataset_2 = CustomDataset(folder_path, 2, delimiter, chunk_size)

# Create DataLoader instances for each dataset
dataloader_1 = DataLoader(dataset_1, batch_size=batch_size, shuffle=True)
dataloader_2 = DataLoader(dataset_2, batch_size=batch_size, shuffle=True)

client_dataloaders = [dataloader_1, dataloader_2]

total = 0
for batch in dataloader_1:
    total += 1
print(total)

#for batch in data_loader:
#    print(batch)

Code samples in partition 1: 3002
Code samples in partition 2: 3004
188


## Pretrained model

This model [name] from Hugging Face is pretrained on code.

In [6]:
## TODO: Jun
## Download model
# Load the CodeParrot small model and tokenizer

from helpers.train import generic_training_runner
from transformers import BitsAndBytesConfig
from peft import LoraConfig

model_name = "codeparrot/codeparrot-small"
run_title = "codeparrotsm"


SAVE_PATH = f"./ronit_ibm_outputs/checkpoints/{run_title}"
LOSSES_PATH = f"./ronit_ibm_outputs/logs/losses/{run_title}"
TIMES_PATH = f"./ronit_ibm_outputs/logs/times/{run_title}_elapsed_time"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model_info = {
    'model_name': model_name,
    'tokenizer': tokenizer,
    'client_dataloaders': client_dataloaders,
    'quant_config':
        BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
    'lora_config':
        LoraConfig(
            r=8,
            target_modules=["c_attn", "c_proj", "c_fc"],
            lora_alpha=32,
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM",
        ),
}

training_args = {
    'clients': 2,
    'data_per_client': 1e3,
    'MAX_LENGTH': 1024,
    'conduct_logging': True,
    'EPOCHS': 1,
    'lr': 4e-4
}


tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

## Partition Code [outdated]

In [ ]:
# ## Partition Dataset

# import random

# # Assuming you have 'n_partitions' as the number of partitions you want
# n_partitions = 5  # for example, change this based on your requirement

# # Shuffle the dataset
# random.shuffle(custom_dataset.script_files)

# # Calculate the size of each partition
# partition_size = len(custom_dataset) // n_partitions

# # Create partitions
# partitions = [custom_dataset.script_files[i * partition_size:(i + 1) * partition_size] for i in range(n_partitions)]

# # Now create a DataLoader for each partition
# dataloaders = []
# for partition in partitions:
#     # Creating a new CustomDataset instance for each partition
#     partition_dataset = CustomDataset(folder_path, delimiter, chunk_size)
#     partition_dataset.script_files = partition  # replace the script_files with the partition

#     # Create a DataLoader for the partitioned dataset
#     partition_dataloader = DataLoader(partition_dataset, batch_size=1, shuffle=True)
#     dataloaders.append(partition_dataloader)

# # Now 'dataloaders' contains separate DataLoader for each partition

# import hashlib
# from itertools import islice

# def hash_code(data):
#     # Create a hash for the given data
#     return hashlib.sha256(str(data).encode()).hexdigest()

# # Iterate through each DataLoader
# for i, each_data_loader in enumerate(dataloaders):
#     print(f"\nDataLoader {i + 1}:")

#     # Iterate through the first 5 batches of each DataLoader
#     for j, batch in enumerate(islice(each_data_loader, 5), 1):
#         # Compute the hash of the batch
#         batch_hash = hash_code(batch)

#         print(f"  Batch {j} Hash: {batch_hash}")





DataLoader 1:


TypeError: ignored

In [ ]:
# # Validation Script

# # Check for equal size
# partition_sizes = [len(partition) for partition in partitions]
# print("Sizes of each partition:", partition_sizes)

# # Check if sizes are approximately equal
# # Note: They might not be exactly equal if the total size is not perfectly divisible by n_partitions
# if all(abs(size - partition_size) <= 1 for size in partition_sizes):
#     print("All partitions are of approximately equal size.")
# else:
#     print("Warning: Partitions sizes vary more than expected.")

# # Consistency Check
# # Example: Checking if each partition still contains script files and not something else
# for i, partition_dataset in enumerate(dataloaders):
#     is_consistent = all(isinstance(script, str) and script.endswith('.py') for script in partition_dataset.dataset.script_files)
#     print(f"Partition {i} consistency: {'Consistent' if is_consistent else 'Inconsistent'}")

# # Note: Replace the consistency check as per your criteria


# def hash_code(code_snippet):
#     # Using SHA-256 for hashing
#     return hashlib.sha256(code_snippet.encode()).hexdigest()

# # Check for duplicates within each partition
# for i, partition in enumerate(partitions):
#     hash_set = set()
#     duplicates = set()
#     for snippet in partition:
#         snippet_hash = hash_code(snippet)
#         if snippet_hash in hash_set:
#             duplicates.add(snippet_hash)
#         else:
#             hash_set.add(snippet_hash)

#     if duplicates:
#         print(f"Partition {i} has duplicates: {duplicates}")
#     else:
#         print(f"Partition {i} has no duplicates.")

# # Check for duplicates across partitions
# global_hash_set = set()
# cross_partition_duplicates = set()
# for partition in partitions:
#     for snippet in partition:
#         snippet_hash = hash_code(snippet)
#         if snippet_hash in global_hash_set:
#             cross_partition_duplicates.add(snippet_hash)
#         else:
#             global_hash_set.add(snippet_hash)

# if cross_partition_duplicates:
#     print(f"Cross-partition duplicates found: {cross_partition_duplicates}")
# else:
#     print("No cross-partition duplicates found.")




Sizes of each partition: [160, 160, 160, 160, 160]
All partitions are of approximately equal size.
Partition 0 consistency: Consistent
Partition 1 consistency: Consistent
Partition 2 consistency: Consistent
Partition 3 consistency: Consistent
Partition 4 consistency: Consistent
Partition 0 has no duplicates.
Partition 1 has no duplicates.
Partition 2 has no duplicates.
Partition 3 has no duplicates.
Partition 4 has no duplicates.
No cross-partition duplicates found.


## **Deduplication** [outdated]

In [ ]:
#dataset = load_dataset("codeparrot/codeparrot-clean")

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
#########not enough RAM -- try process in chunks ??????????
path = '/content/drive/MyDrive/CS6220 Folder/codeparrot-clean/extracted/'

import json
import pandas as pd

all_code_content = []

for i in range(1, 55):
    file_name = f'file-{i:012}.json'
    full_path = f'{path}{file_name}'
    try:
        with open(full_path, 'r') as file:
            for line in file:
                try:
                    data = json.loads(line)
                    code_content = data.get('content', '')
                    if code_content:
                        all_code_content.append({'content': code_content})
                except json.JSONDecodeError as e:
                    print(f"JSON decode error in file {file_name}, line: {line}")
    except FileNotFoundError as e:
        print(f"File not found: {e}")

df = pd.DataFrame(all_code_content)



KeyboardInterrupt: ignored

## **Big Code/ The-Stack** [outdated]

In [ ]:
!pip install huggingface-cli
!huggingface-cli login


In [ ]:
from  datasets  import  load_dataset

# full dataset (3TB of data)
#ds = load_dataset("bigcode/the-stack", split="train")

# specific language (e.g. Dockerfiles)
ds = load_dataset("bigcode/the-stack", data_dir="data/python", split="train")

# dataset streaming (will only download the data as needed)
#ds = load_dataset("bigcode/the-stack", streaming=True, split="train")
#for sample in iter(ds): print(sample["content"])


##run script

In [10]:
generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_info,
    training_args,
)

Device Type: cuda
Beginning training iteration
Loaded global model.
Number of batches: 188
Starting training


1it [00:01,  1.92s/it]

Step: 0, Loss: 8.2141


2it [00:04,  2.23s/it]

Step: 1, Loss: 6.0871


...

Step: 185, Loss: 0.4195


187it [05:38,  1.32s/it]

Step: 186, Loss: 0.4842


188it [05:40,  1.81s/it]

Step: 187, Loss: 0.3386
Epoch 0 Complete
Ending training
340.0854561328888
MODEL SAVED


In [31]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

import torch

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

import pickle

import time
from tqdm import tqdm


def generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_info,
    training_args,
):
    """
    Function to perform finetuning on provided model_name from Hugging Face

    Models are checkpointed and saved at provided locations in Drive.

    :param str SAVE_PATH: Location to save model checkpoint
    :param str LOSSES_PATH: Location to log loss results
    :param str TIMES_PATH: Location to log elapsed time results
    :param model_info: Object containing information about model
    :param training_args: Object containing training arguments
    :return: None
    """
    # Args
    clients, MAX_LENGTH, conduct_logging, EPOCHS, lr = (
        training_args['clients'],
        training_args['MAX_LENGTH'],
        training_args['conduct_logging'],
        training_args['EPOCHS'],
        training_args['lr'],
    )
    model_name, tokenizer, client_dataloaders, quant_config, lora_config = (
        model_info['model_name'],
        model_info['tokenizer'],
        model_info['client_dataloaders'],
        model_info['quant_config'],
        model_info['lora_config'],
    )

    assert len(client_dataloaders) == clients

    # Device setup
    torch.manual_seed(42)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device Type: {device}")


    # Loggings of losses setup.
    def write_to_file(epoch_number, loss_values, avg_loss, file_path):
        # Open the file in binary append mode
        with open(file_path + ".txt", "ab") as file:
            # Serialize and write the epoch number
            pickle.dump(f"Epoch {epoch_number}", file)
            # Serialize and write the list of loss values
            pickle.dump(loss_values, file)
            # Average loss for the epoch
            pickle.dump(avg_loss, file)

    # Simple test to log.
    conduct_logging = True
    if conduct_logging:
        write_to_file(-1, [-1, -2, -3], [-2], LOSSES_PATH)

    # Begin training process
    # Traditional training loop requires modifications for FL...
    # To do this, we perform the following. Assume only 1 epoch is done.
    """
    Create a variable to aggregate gradients

    For Each client
        Copy the global model
        Fetch the client's dataloader
        Train across the data in one iteration.
        Log losses carefully
        Save model
        Aggregate change in weights to variable
    """

    def train_loop(client_idx):
        """
        Main training loop

        :param int client_idx: The index of the client being trained
        :return: None
        """
        global_model = AutoModelForCausalLM.from_pretrained(
            model_name, quantization_config=quant_config, device_map={"": 0}
        )
        global_model.gradient_checkpointing_enable()

        global_model = prepare_model_for_kbit_training(global_model)
        global_model = get_peft_model(global_model, lora_config).to(device)
        client_model = global_model
        print("Loaded global model.")

        # Set up loss and optimization unique to model.
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(client_model.parameters(), lr=lr)

        losses = []
        # Log a 10K losses
        dataloader_client = client_dataloaders[client_idx]
        log_every = max(1, (EPOCHS * len(dataloader_client)) // 10_000)
        print("Number of batches:", len(dataloader_client))
        client_model.train()
        start_time = time.time()
        print("Starting training")
        for epoch in range(EPOCHS):
            total_loss = 0
            for c, batch in tqdm(enumerate(dataloader_client)):
                # Batch size x Max Seq LEn
                sample = tokenizer(
                    batch,
                    padding=True,
                    truncation=True,
                    return_tensors="pt",
                    max_length=MAX_LENGTH,
                )["input_ids"].to(device)
                target = sample.detach()[:, 1:]
                sample = sample[:, :-1]

                # Batch size x Max Seq Len x Vocab Size
                optimizer.zero_grad()
                prediction = client_model(sample).logits

                # Ensure swapping of axes
                loss = criterion(prediction.transpose(1, 2), target)
                loss.backward()

                # Loss logging
                total_loss += loss.item()
                if c % log_every == 0:
                    print(f"Step: {c}, Loss: {loss.item():.4f}")
                    losses.append(loss.item())

                # Change model weights
                optimizer.step()

                # Explicit destruction (may not be needed after previous debugging)
                del loss, prediction, sample, target
                torch.cuda.empty_cache()

            print(f"Epoch {epoch} Complete")
            avg_loss = total_loss / len(dataloader_client)
            if conduct_logging:
                write_to_file(epoch, losses, avg_loss, LOSSES_PATH + f"_{client_idx}")
            losses.clear()

        print("Ending training")

        end_time = time.time()

        optimizer.zero_grad()

        elapsed_time = end_time - start_time
        if conduct_logging:
            with open(TIMES_PATH + f"_{client_idx}.txt", "a") as file:
                file.write(f"{elapsed_time}")

        print(elapsed_time)
        if conduct_logging:
            client_model.save_pretrained(SAVE_PATH + f"_{client_idx}")
            print("MODEL SAVED")
            return client_model
            del client_model
            torch.cuda.empty_cache()

    # Execute training for all clients
    client_rets = []
    for i in range(clients):
        print("Beginning training iteration")
        client_rets.append(train_loop(i))
    return client_rets

client_rets = generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_info,
    training_args,
)

Device Type: cuda
Beginning training iteration
Loaded global model.
Number of batches: 188
Starting training


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
1it [00:01,  1.89s/it]

Step: 0, Loss: 8.2141


2it [00:04,  2.18s/it]

Step: 1, Loss: 6.0871


...

Step: 185, Loss: 0.4195


187it [05:45,  1.35s/it]

Step: 186, Loss: 0.4842


188it [05:47,  1.85s/it]

Step: 187, Loss: 0.3386
Epoch 0 Complete
Ending training
347.58215522766113
MODEL SAVED


In [25]:
importlib.reload(helpers)

from helpers.fl_impl import get_fed_avg_model
from peft import prepare_model_for_kbit_training, get_peft_model

SAVE_PATH = f"./ronit_ibm_outputs/checkpoints/{run_title}"

global_aggregated_model = get_fed_avg_model(SAVE_PATH, model_info, num_clients=2)
torch.cuda.empty_cache()

Device Type: cuda
--------------------------------
Processing client 0
Loading model from ./ronit_ibm_outputs/checkpoints/codeparrotsm_0
Loaded model


AttributeError: ignored

In [34]:
# Dealing with above bug in peft library
weight_dict = {}
for client_idx, client in enumerate(client_rets):
    for k,v in client.state_dict().items():
        if isinstance(v, str):
                weight_dict[k] = v
        else:
            if client_idx == 0:       # For the first client, we need to initialize the value.
                weight_dict[k] = 0
            weight_dict[k] += v/torch.tensor(2)
aggregated_model = AutoModelForCausalLM.from_pretrained(
    model_info['model_name'], quantization_config=model_info['quant_config'], device_map={"": 0}
)
aggregated_model = prepare_model_for_kbit_training(aggregated_model)
global_aggregated_model = get_peft_model(aggregated_model, model_info['lora_config']).to('cuda')

global_aggregated_model.load_state_dict(weight_dict)

<All keys matched successfully>

In [35]:
import importlib

import helpers
import helpers.evaluation
importlib.reload(helpers)
importlib.reload(helpers.evaluation)

from helpers.evaluation import (
    generate_batch_completions,
    get_humaneval_dataloader,
    generate_model_on_problems,
    get_prompt_from_descr,
    clean_samples,
    compute_average_rouge_scores,
    print_gens_on_problem_all_models,
)

In [36]:
import human_eval
importlib.reload(human_eval)
from human_eval.data import write_jsonl, read_problems

problems = read_problems()
probs_dataloader = get_humaneval_dataloader(problems, batch_size=10)

In [37]:
samples = generate_model_on_problems(global_aggregated_model, probs_dataloader, tokenizer)

write_jsonl("human_eval/samples-sf.json", samples)

Device Type: cuda
Generated completions for batch 0 of 17
Device Type: cuda
Generated completions for batch 1 of 17
Device Type: cuda
Generated completions for batch 2 of 17
Device Type: cuda
Generated completions for batch 3 of 17
Device Type: cuda
Generated completions for batch 4 of 17
Device Type: cuda
Generated completions for batch 5 of 17
Device Type: cuda
Generated completions for batch 6 of 17
Device Type: cuda
Generated completions for batch 7 of 17
Device Type: cuda
Generated completions for batch 8 of 17
Device Type: cuda
Generated completions for batch 9 of 17
Device Type: cuda
Generated completions for batch 10 of 17
Device Type: cuda
Generated completions for batch 11 of 17
Device Type: cuda
Generated completions for batch 12 of 17
Device Type: cuda
Generated completions for batch 13 of 17
Device Type: cuda
Generated completions for batch 14 of 17
Device Type: cuda
Generated completions for batch 15 of 17
Device Type: cuda
Generated completions for batch 16 of 17


In [45]:
!python3 human_eval/evaluate_functional_correctness.py human_eval/samples-sf.json

Reading samples...
164it [00:00, 31829.43it/s]
Running test suites...
100% 164/164 [00:01<00:00, 126.86it/s]
Writing results to human_eval/samples-sf.json_results.jsonl...
100% 164/164 [00:00<00:00, 25607.40it/s]
{'pass@1': 0.0}


In [46]:
references = [problems[f'HumanEval/{i}']['prompt'] for i in range(len(samples))]
hypotheses = [samples[i]['completion'] for i in range(len(samples))]

# Compute average ROUGE scores
average_rouge_scores = compute_average_rouge_scores(references, hypotheses)
print(average_rouge_scores)

{'rouge-1': {'f': 0.181439196656887, 'p': 0.19444999640380556, 'r': 0.19924353054708221}, 'rouge-2': {'f': 0.02186178882053857, 'p': 0.02247974187441289, 'r': 0.0255706528152988}, 'rouge-l': {'f': 0.16800162708168045, 'p': 0.18028544477288624, 'r': 0.18456777342031117}}


In [ ]:
print_gens_on_problem_all_models(
    SAVE_PATH,
    "/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p00003.html",
    model_info,
    global_aggregated_model
)

In [48]:
samples = generate_model_on_problems(client_rets[0], probs_dataloader, tokenizer)

write_jsonl("human_eval/samples-client0.json", samples)

!python3 human_eval/evaluate_functional_correctness.py human_eval/samples-client0.json

references = [problems[f'HumanEval/{i}']['prompt'] for i in range(len(samples))]
hypotheses = [samples[i]['completion'] for i in range(len(samples))]

# Compute average ROUGE scores
average_rouge_scores = compute_average_rouge_scores(references, hypotheses)
print(average_rouge_scores)

Device Type: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Generated completions for batch 0 of 17
Device Type: cuda
Generated completions for batch 1 of 17
Device Type: cuda
Generated completions for batch 2 of 17
Device Type: cuda
Generated completions for batch 3 of 17
Device Type: cuda
Generated completions for batch 4 of 17
Device Type: cuda
Generated completions for batch 5 of 17
Device Type: cuda
Generated completions for batch 6 of 17
Device Type: cuda
Generated completions for batch 7 of 17
Device Type: cuda
Generated completions for batch 8 of 17
Device Type: cuda
Generated completions for batch 9 of 17
Device Type: cuda
Generated completions for batch 10 of 17
Device Type: cuda
Generated completions for batch 11 of 17
Device Type: cuda
Generated completions for batch 12 of 17
Device Type: cuda
Generated completions for batch 13 of 17
Device Type: cuda
Generated completions for batch 14 of 17
Device Type: cuda
Generated completions for batch 15 of 17
Device Type: cuda
Generated completions for batch 16 of 17
Reading samples...
164it 

In [52]:
prompt = get_prompt_from_descr("/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p00003.html")
print(generate_batch_completions([prompt], client_rets[0], tokenizer)[0])
print(generate_batch_completions([prompt], client_rets[1], tokenizer)[0])
print(generate_batch_completions([prompt], global_aggregated_model, tokenizer)[0])

Device Type: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


"""

Is it a Right Triangle?

Write a program which judges wheather given length of three side form a right triangle. Print "YES" if the given sides (integers) form a right triangle, "NO" if not so.


"""

def main():
    n = int(input())
    if n == 1:
        print("NO")
    elif n == 2:
        print("YES")
    elif n == 3:
        print("NO")
    else:
        print("YES")



Device Type: cuda
"""

Is it a Right Triangle?

Write a program which judges wheather given length of three side form a right triangle. Print "YES" if the given sides (integers) form a right triangle, "NO" if not so.


"""

def main():
    n = int(input())
    if n < 10:
        print(input())
    else:
        print(input())


Device Type: cuda
"""

Is it a Right Triangle?

Write a program which judges wheather given length of three side form a right triangle. Print "YES" if the given sides (integers) form a right triangle, "NO" if not so.


"""

def main():
    print("Yes")




In [49]:
samples = generate_model_on_problems(client_rets[1], probs_dataloader, tokenizer)

write_jsonl("human_eval/samples-client0.json", samples)

!python3 human_eval/evaluate_functional_correctness.py human_eval/samples-client0.json

references = [problems[f'HumanEval/{i}']['prompt'] for i in range(len(samples))]
hypotheses = [samples[i]['completion'] for i in range(len(samples))]

# Compute average ROUGE scores
average_rouge_scores = compute_average_rouge_scores(references, hypotheses)
print(average_rouge_scores)

Device Type: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Generated completions for batch 0 of 17
Device Type: cuda
Generated completions for batch 1 of 17
Device Type: cuda
Generated completions for batch 2 of 17
Device Type: cuda
Generated completions for batch 3 of 17
Device Type: cuda
Generated completions for batch 4 of 17
Device Type: cuda
Generated completions for batch 5 of 17
Device Type: cuda
Generated completions for batch 6 of 17
Device Type: cuda
Generated completions for batch 7 of 17
Device Type: cuda
Generated completions for batch 8 of 17
Device Type: cuda
Generated completions for batch 9 of 17
Device Type: cuda
Generated completions for batch 10 of 17
Device Type: cuda
Generated completions for batch 11 of 17
Device Type: cuda
Generated completions for batch 12 of 17
Device Type: cuda
Generated completions for batch 13 of 17
Device Type: cuda
Generated completions for batch 14 of 17
Device Type: cuda
Generated completions for batch 15 of 17
Device Type: cuda
Generated completions for batch 16 of 17
Reading samples...
164it 